In [2]:
from sage.all import *
import hashlib

class BF_Cryptosystem():

    def __init__(self, lmbda, nounce):
        self.lmbda = lmbda
        self.bq = 2 ^ (self.lmbda - 1)
        self.bp = 2 ^ self.lmbda - 1  
        self.q = random_prime(self.bp, lbound = self.bq)
        self.nounce = nounce


        t = self.q * 3 * 2 ^ (self.bp - self.bq)
        while not (t - 1).is_prime():
            t = t << 1

        self.p = t - 1
        Fp = GF(self.p)
        R.<z> = Fp[]
        f = R(z ^ 2 + z + 1)
        Fp2.<z> = GF(self.p ^ 2, modulus=f)
        self.z = z
        self.E2 = EllipticCurve(Fp2, [0,1])

        cofac = (self.p + 1) // self.q
        self.G = cofac * self.E2.random_point()

    
    def phi(self, P):
        (x, y) = P.xy()

        return self.E2(self.z * x, y)


    def ex(self, P, Q, l=1):
        return P.tate_pairing(self.phi(Q), self.q, 2) ^ l


    def Zr(self):
        encoded_nounce = str(self.nounce).encode()
        hashed_nounce = hashlib.sha256(encoded_nounce).digest()
        int_hashed_nounce = int.from_bytes(hashed_nounce, byteorder='big') % self.q

        return int_hashed_nounce
    

    def f(self, P):
        x, y = P[0], P[1]

        data = str(x) + str(y)
        hashed_data = hashlib.sha256(data.encode()).hexdigest()
        hash_integer = Integer(int(hashed_data, 16))
    
        return hash_integer

    

    def h(self, bts):
        hash_object = hashlib.sha256()
        hash_object.update(bts)
        hex_hash = hash_object.hexdigest()

        return Integer('0x' + hex_hash)


    def H(self, z):
        encoded_z = str(z).encode()
        hashed_z = hashlib.sha256(encoded_z).digest()
        int_hashed_z = int.from_bytes(hashed_z, byteorder='big') % self.q

        return int_hashed_z

    
    def g(self, s):
        return s * self.G
    

    def id(self, bts):
        return self.g(self.h(bts))


    def keygen(self):
        s = self.Zr()
        beta = self.g(s)

        return s, beta
    

    def keyextract(self, id, s):
        d = self.id(id)

        return s * d
    

    def in_encrypt(self, id, x, beta):
        d = self.id(id)
        v = self.Zr()
        a = self.H(v ^^ x)
        u = self.ex(beta, d, a)

        return x, v, a, u
    
    
    def out_encrypt(self, x, v, a, u):
        alpha = self.g(a)
        vl = v ^^ self.f(u)
        xl = x ^^ self.H(v)

        return alpha, vl, xl


    def encrypt(self, id, x, beta):
        x, v, a, u = self.in_encrypt(id, x, beta)

        return self.out_encrypt(x, v, a, u)
    

    def in_decrypt(self, alpha, vl, xl, key):
        u = self.ex(alpha, key, 1)
        v = vl ^^ self.f(u)
        x = xl ^^ self.H(v)

        return alpha, v, x
    

    def out_decrypt(self, alpha, v, x):
        a = self.H(v ^^ x)
        if alpha != self.g(a):
            return None
        
        return x
    

    def decrypt(self, key, alpha, vl, xl):
        alpha, v, x = self.in_decrypt(alpha, vl, xl, key)
        result = self.out_decrypt(alpha, v, x)

        if result == 'None':
            print('[ERROR] decryption failed')
        else:
            print(f'[CORRECT DECRYPTION] {result}')
        


bf_cs = BF_Cryptosystem(4, 12345677654321)
s, beta = bf_cs.keygen()
id = b'1231232131'
key = bf_cs.keyextract(id, s)


x = 12345

alpha, vl, xl = bf_cs.encrypt(id, x, beta)
bf_cs.decrypt(key, alpha, vl, xl)

[CORRECT DECRYPTION] 12345


# Ricardo Mandaram me este resolução, amanhã em chamada vemos isto.
AINDA faltam corrrgit algumas cenasSs

tipo o 
Zr
se reparares ela usa sempre o mesmo nounce
acho que é melhor utilizar o nounce random cada vez que ela é chamada